## Transfer Learning
Author: Braulio Otavalo

In [7]:
#!pip install tensorflow
#!pip install kagglehub

Import datadet from Kaggle

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")

print("Path to dataset files:", path)

/opt/conda/envs/tl_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 787M/787M [00:10<00:00, 79.7MB/s] 

Extracting files...


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/karakaggle/kaggle-cat-vs-dog-dataset/versions/1


In [8]:
"""import os
import shutil
import random

# Define the source and destination directories
source_dirs = ['/workspaces/transfer-learning/data/Cat', '/workspaces/transfer-learning/data/Dog']
train_dirs = ['/workspaces/transfer-learning/data/train/Cat', '/workspaces/transfer-learning/data/train/Dog']
validation_dirs = ['/workspaces/transfer-learning/data/validation/Cat', '/workspaces/transfer-learning/data/validation/Dog']

# Create the destination directories if they don't exist
for dir in train_dirs + validation_dirs:
    os.makedirs(dir, exist_ok=True)

# Function to split files into train and validation sets
def split_files(source_dir, train_dir, validation_dir, split_ratio=0.8):
    files = os.listdir(source_dir)
    random.shuffle(files)
    split_index = int(len(files) * split_ratio)
    train_files = files[:split_index]
    validation_files = files[split_index:]

    for file in train_files:
        shutil.move(os.path.join(source_dir, file), os.path.join(train_dir, file))
    for file in validation_files:
        shutil.move(os.path.join(source_dir, file), os.path.join(validation_dir, file))

# Split the files for each category
for source_dir, train_dir, validation_dir in zip(source_dirs, train_dirs, validation_dirs):
    split_files(source_dir, train_dir, validation_dir)"""

In [9]:
import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

categories = ['Cat', 'Dog']
data_types = ['train', 'validation']

for data_type in data_types:
    for category in categories:
        dir_path = f'/workspaces/transfer-learning/data/{data_type}/{category}'
        num_files = count_files(dir_path)
        print(f'Number of {category} images in {data_type}: {num_files}')

Number of Cat images in train: 9992
Number of Dog images in train: 9976
Number of Cat images in validation: 2499
Number of Dog images in validation: 2494


In [10]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the VGG16 model pre-trained on ImageNet, without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add new classification layers on top of the pre-trained model
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2025-02-08 03:10:54.371788: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-08 03:10:54.619914: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-08 03:10:54.740448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738984255.077438   17213 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738984255.168880   17213 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 03:10:55.852863: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/workspaces/transfer-learning/data/train', target_size=(224, 224), batch_size=32, class_mode='binary')
validation_generator = test_datagen.flow_from_directory('/workspaces/transfer-learning/data/validation', target_size=(224, 224), batch_size=32, class_mode='binary')

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Found 19966 images belonging to 2 classes.
Found 4993 images belonging to 2 classes.
Epoch 1/10
